In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

# Define Actor and Critic networks
class Actor(nn.Module):
    def __init__(self, input_size, output_size):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Critic(nn.Module):
    def __init__(self, input_size):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define Actor-Critic agent
class ActorCriticAgent:
    def __init__(self, env):
        self.env = env
        self.observation_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n

        # Initialize actor and critic networks
        self.actor = Actor(self.observation_size, self.action_size)
        self.critic = Critic(self.observation_size)

        # Initialize optimizer
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=0.001)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=0.001)

    def select_action(self, state):
        state = torch.FloatTensor(state)
        action_probs = F.softmax(self.actor(state), dim=-1)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()
        return action.item()

    def update(self, state, action, reward, next_state, done):
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        action = torch.LongTensor([action])
        reward = torch.FloatTensor([reward])

        # Compute state value estimates
        state_value = self.critic(state)
        next_state_value = self.critic(next_state)
        if done:
            target_value = reward
        else:
            target_value = reward + next_state_value

        # Compute advantages
        advantage = target_value - state_value

        # Update critic network
        critic_loss = advantage.pow(2).mean()
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Update actor network using Proximal Policy Optimization (PPO) objective
        action_probs = F.softmax(self.actor(state), dim=-1)
        old_action_dist = Categorical(action_probs)
        old_log_prob = old_action_dist.log_prob(action)

        for _ in range(10):
            new_action_probs = F.softmax(self.actor(state), dim=-1)
            new_action_dist = Categorical(new_action_probs)
            new_log_prob = new_action_dist.log_prob(action)
            ratio = torch.exp(new_log_prob - old_log_prob)

            surrogate1 = ratio * advantage
            surrogate2 = torch.clamp(ratio, 1 - 0.2, 1 + 0.2) * advantage
            actor_loss = -torch
